<a target="_blank" href="https://colab.research.google.com/github/AndyRids/ipymediator/blob/main/examples/ipymediator_examples.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

In [ ]:
!pip install git+https://github.com/AndyRids/ipymediator.git@main

In [ ]:
from ipymediator.enumerations import Value
from ipymediator.interface import Mediator, Component
from ipymediator.utils import singlenotifydispatch
from ipywidgets import widgets as w
from traitlets import HasTraits, traitlets

In [ ]:
class Dialog(Mediator, HasTraits):
    """Receives messages from Button and Text widgets and passes an action
    to carry out, to the appropriate widget on message receipt"""

    button_counter = traitlets.Integer(default_value=0, help="button clicks").tag(config=True)

    def __init__(self):
        super(Dialog, self).__init__()

        # Component adds Bool trait called value to all Button widgets by default,
        # which toggles True/False on each click (like ToggleButton widget)
        self.button_submit = Component(
            mediator=self, widget=w.Button(), widget_name="button_submit", names=("value",))

        # Component __init__ sets "value" trait to be observed by default
        self.message_clicks = Component(
            mediator=self, widget=w.Text(), widget_name="message_clicks")
        
        self.message_value = Component(
            mediator=self, widget=w.Text(), widget_name="message_value", names=("disabled",))

        # Component allows changes to widget property traits through 
        # bracket notation (see Component info for details)
        self.button_submit["description"] = "Click Me"
        self.button_submit["layout"].width = "300px"
        self.button_submit["style"].font_weight = "bold"
        self.button_submit["style"].button_color = "#f8edeb"

        self.message_clicks["style"].width = "300px"
        self.message_value["style"].width = "300px"

        self.container = w.VBox(
            children=(
                self.button_submit.widget,
                self.message_clicks.widget,
                self.message_value.widget
            )
        )

    @singlenotifydispatch
    def notify(self, reference: str, change: Value) -> None:
        """Receives nessages from concrete Components
        
        Params:
            reference (str): Reference value passed by Component. Can be used to 
                differentiate message origin.
            
            change (Value): trait value change dict passed by traitlets observe function.
        """
        pass
           
    @notify.register("button_submit")
    def _(self, reference: str, change: Value) -> None:
        # button_submit.widget was clicked
        self.button_counter += 1
        self.message_clicks["value"] = f"Button clicks: {self.button_counter}"
        self.message_value["value"] = f"Button value is {self.button_submit['value']}"
        self.message_value["disabled"] = self.button_submit["value"]

    @notify.register("message_value")
    def _(self, reference: str, change: Value) -> None:
        # message_value.widget disabled trait change
        message = ("enabled", "disabled")[int(change["new"])]
        print(f"message_value.widget was {message}")

In [ ]:
dialog = Dialog()
dialog.container